In [92]:
from tensorflow.keras.layers import Input, Dense, Softmax
from tensorflow.keras import Model
import numpy as np

In [144]:
inputs = Input(shape=(28*28,))
out = Dense(56, activation='relu')(inputs)
out = Dense(10)(out)
out = Softmax()(out)
model_tf = Model(inputs, out)

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [145]:
model_tf.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 784)]             0         
                                                                 
 dense_31 (Dense)            (None, 56)                43960     
                                                                 
 dense_32 (Dense)            (None, 10)                570       
                                                                 
 softmax_5 (Softmax)         (None, 10)                0         
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [146]:
import tensorflow as tf
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

In [147]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=10, batch_size=256, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
211/211 [==============================] - 1s 1ms/step - loss: 0.6104 - accuracy: 0.8349 - val_loss: 0.2516 - val_accuracy: 0.9335
Epoch 2/10
211/211 [==============================] - 0s 957us/step - loss: 0.2680 - accuracy: 0.9249 - val_loss: 0.1886 - val_accuracy: 0.9515
Epoch 3/10
211/211 [==============================] - 0s 1ms/step - loss: 0.2126 - accuracy: 0.9403 - val_loss: 0.1600 - val_accuracy: 0.9560
Epoch 4/10
211/211 [==============================] - 0s 1ms/step - loss: 0.1770 - accuracy: 0.9499 - val_loss: 0.1412 - val_accuracy: 0.9635
Epoch 5/10
211/211 [==============================] - 0s 1ms/step - loss: 0.1527 - accuracy: 0.9568 - val_loss: 0.1254 - val_accuracy: 0.9655
Epoch 6/10
211/211 [==============================] - 0s 1ms/step - loss: 0.1352 - accuracy: 0.9617 - val_loss: 0.1194 - val_accuracy: 0.9658
Epoch 7/10
211/211 [==============================] - 0s 1ms/step - loss: 0.1214 - accuracy: 0.9652 - val_loss: 0.1152 - val_accuracy: 0.9673
Epoc

## Convert

In [148]:
model_tf.save("test.h5")

/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [149]:
command = ['python','main.py', "./test.h5", "-o", "./test_model/"]
import subprocess
subprocess.run(command)

2024-02-01 11:28:30.986095: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 11:28:31.005952: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 11:28:31.005974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 11:28:31.006476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 11:28:31.009511: I tensorflow/core/platform/cpu_feature_guar

CompletedProcess(args=['python', 'main.py', './test.h5', '-o', './test_model/'], returncode=1)

## Load and Test

In [150]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
X = test_images_tf[4]
#X = np.random.rand(1,784)[0]

X_in = [int(x*1e36) for x in X]

In [151]:
model_tf.weights[0].shape

TensorShape([784, 56])

#### New

In [152]:

Dense32weights = [[int(model_tf.weights[0].numpy()[i][j]*1e36) for j in range(56)] for i in range(784)]
Dense32bias = [int(model_tf.weights[1].numpy()[i]*1e72) for i in range(56)]

Dense21weights = [[int(model_tf.weights[2].numpy()[i][j]*1e36) for j in range(10)] for i in range(56)]
Dense21bias = [int(model_tf.weights[3].numpy()[i]*1e72) for i in range(10)]

In [153]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [154]:
X_in, Dense32weights, Dense32bias, Dense32out, Dense32remainder = DenseInt(784, 56, 10**36, X_in, Dense32weights, Dense32bias)
Dense32out, Dense32remainder

(['21888242871839275222246405745257275088548322211608623854345287816239773308690',
  '838127788734923734202849044168451719',
  '989682002116323459865011092253239033',
  '21888242871839275222246405745257275088547837005085880353600835772829491813172',
  '21888242871839275222246405745257275088546681325947829007611960439691397236159',
  '70306491376558188082876180923208898',
  '2653387504983088696145886993282555768',
  '2897930119007223441137461037794890229',
  '21888242871839275222246405745257275088548264352036519666791453960988047706102',
  '21888242871839275222246405745257275088548308234756370560494393777332216095867',
  '2311884288251509802362091076182681762',
  '21888242871839275222246405745257275088547393936530198113461394877447896151659',
  '1018971716778754653207269380342185146',
  '21888242871839275222246405745257275088548343227263662066155831205117888076463',
  '21888242871839275222246405745257275088546360966396581652067013953155575587023',
  '259612088331875095859518320686850497

In [166]:
2**18 - 73416*2

115312

: 

In [155]:
ReLUout = [Dense32out[i] if int(Dense32out[i]) < p//2 else 0 for i in range(56)]
Dense21in = [int(ReLUout[i]) for i in range(56)]

In [156]:
_, Dense21weights, Dense21bias, Dense21out, Dense21remainder = DenseInt(56, 10, 10**36, Dense21in, Dense21weights, Dense21bias)
Dense21out, Dense21remainder

(['21888242871839275222246405745257275088546063345820156910217155069369018194114',
  '21888242871839275222246405745257275088538590289104506794594989949241573741180',
  '21888242871839275222246405745257275088546103770114443425951061204561191726538',
  '21888242871839275222246405745257275088540742419736991881987614813282035171350',
  '5641281201191648344837512235207434840',
  '21888242871839275222246405745257275088541824554966492621219814005755554337498',
  '21888242871839275222246405745257275088548160952719083071203795248972577137373',
  '21888242871839275222246405745257275088545629352270692811625938920666603684864',
  '21888242871839275222246405745257275088546698678871651554478873733940295618882',
  '118424560718366063795441929229994668'],
 ['335124400235748579544585479145914368',
  '603849222862733966791444371228065792',
  '968872479321219891088160199442169856',
  '702370338190297604758116005299879936',
  '803442790744809603223506146039955456',
  '259368469105901904114611693207683072'

In [160]:
out = int(np.argmax([int(x) for x in ["146056340277142749194949014098804736", "39492307559960320980712927461900288", "738329582115306739978087991762485248", "999573412960859064739198007037132800", "787150261617010841260740606907383808", "354730341639184725959469694472683520", "64983515283906415008334548582793216", "843260069024032851277416705508245504", "324453694733099255386432265024176128", "288508078226094621922833223366213632"]]))

In [161]:
out

3

In [159]:
import json
in_json = {
    "in": X_in,
    "Dense32weights": Dense32weights,
    "Dense32bias": Dense32bias,
    "Dense32out": Dense32out,
    "Dense32remainder": Dense32remainder,
    "ReLUout": ReLUout,
    "Dense21weights": Dense21weights,
    "Dense21bias": Dense21bias,
    "Dense21out": Dense21out,
    "Dense21remainder": Dense21remainder
}

with open("./test_model/dnn_input.json", "w") as f:
    json.dump(in_json, f)

#### Old

In [98]:
dense_5_weights = [[None for _ in range(56)] for _ in range(784)]
for i in range(784):
    for j in range(56):
        dense_5_weights[i][j] = int(model_tf.get_weights()[0][i][j]*1e18)

dense_5_bias = [int(b*1e36) for b in model_tf.get_weights()[1]]

dense_6_weights = [[int(model_tf.weights[2].numpy()[i][j]*1e18) for j in range(10)] for i in range(56)]
dense_6_bias = [int(b*1e36) for b in model_tf.get_weights()[3]]


In [99]:
# inputs = Input(shape=(20,))
# out = Dense(10)(inputs)
# model = Model(inputs, out)

# X = np.random.rand(1,20)
# X_in = [int(x*1e18) for x in X[0]]
# weights = [[None for _ in range(10)] for _ in range(20)]
# for i in range(20):
#     for j in range(10):
#         weights[i][j] = int(model.get_weights()[0][i][j]*1e18)
# bias = [int(b*1e36) for b in model.get_weights()[1]]

In [65]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    print (nInputs, len(input))
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p
            
    return Input, Weights, Bias, out, remainder

In [101]:
X_in_, dense_5_weights_, dense_5_bias_, dense_5_out, dense_5_remainder = DenseInt(784, 56, 10**18, X_in, dense_5_weights, dense_5_bias)
dense_5_out

784 784


[21888242871839275222246405745257275088548364400416034343695413673901769423381,
 21888242871839275222246405745257275088548364400416034343697080305351375549208,
 994954139206503724,
 460690161903381966,
 1441875931380006119,
 2568761130285394505,
 6243527618083844931,
 21888242871839275222246405745257275088548364400416034343697194592768764969683,
 21888242871839275222246405745257275088548364400416034343697153028417008716098,
 21888242871839275222246405745257275088548364400416034343697872696290751213324,
 539200387517099896,
 2761491409260505672,
 21888242871839275222246405745257275088548364400416034343696270106684133325424,
 21888242871839275222246405745257275088548364400416034343697289969882144042096,
 1241712651730003742,
 21888242871839275222246405745257275088548364400416034343697663845024392873425,
 21888242871839275222246405745257275088548364400416034343697680159256366087807,
 1102938592615628635,
 2725362754133803799,
 21888242871839275222246405745257275088548364400416034343696461

In [102]:
_, weights, bias, out, remainder = DenseInt(56, 10, 10**18, dense_5_out, dense_6_weights, dense_6_bias, last_layer=True)
out, remainder

56 56


(['12075379958369603242830399738394312734247806963228711798442092012114202897631',
  '6468580745065499922344663494331649573125265831871662405838953890259293768125',
  '763248891612557251503749184682887381615347598960095945515341839628435635771',
  '592425057279761147925116737871979863078696037190077435003232943214624548938',
  '14099676131393303284942703490550866217556714618328391095276685312335041929832',
  '14588359524946574554739773191281036571493956608434419907220394573887547359948',
  '20785642866932806016285982668887898429952420686350829461302888974244918243721',
  '9797395273936724840545722035256917984029012891693886374417107746960209053066',
  '4260982254807358892295140774482644374077719977189807542578040628126831601434',
  '12880104521965360909146243109399417530551522995703937508086210317168434950636'],
 ['453077510589187119',
  '367820727835617810',
  '916977953582031334',
  '504128091219787531',
  '739066893170160758',
  '514650509222634778',
  '432894158362207454',
  '16341

In [64]:
np.argmax(out)

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='softmax/Softmax:0', description="created by layer 'softmax'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [104]:
import json
in_json = {
    "in": X_in_,
    "dense_5_weights": dense_5_weights,
    "dense_5_bias": dense_5_bias,
    "dense_5_out": dense_5_remainder,
    "dense_5_remainder": dense_5_remainder,


    "dense_6_weights": weights,
    "dense_6_bias": bias,
    "dense_6_out": out,
    "dense_6_remainder": remainder

    "argmax_out": 
}

with open("./test_model/dense_input.json", "w") as f:
    json.dump(in_json, f)